In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/CS546/course-project/SemEval

/content/drive/MyDrive/CS546/course-project/SemEval


In [3]:
import pandas as pd
from tqdm import tqdm
import os
import sys
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.metrics import classification_report as report
from sklearn.feature_extraction.text import CountVectorizer
import argparse
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import MultiLabelBinarizer

In [6]:
def make_dataframe(input_folder, labels_folder=None):
    #MAKE TXT DATAFRAME
    text = []
    
    for fil in tqdm(filter(lambda x: x.endswith('.txt'), os.listdir(input_folder))):
        
        iD, txt = fil[7:].split('.')[0], open(input_folder + '/' + fil, 'r', encoding='utf-8').read()
        text.append((iD, txt))

    df_text = pd.DataFrame(text, columns=['id','text']).set_index('id')
    df = df_text
    
    #MAKE LABEL DATAFRAME
    if labels_folder:
        labels = pd.read_csv(labels_folder, sep='\t', header=None)
        labels = labels.rename(columns={0:'id',1:'frames'})
        labels.id = labels.id.apply(str)
        labels = labels.set_index('id')

        #JOIN
        df = labels.join(df_text)[['text','frames']]
        
    
    return df

In [7]:
folder_train = "/content/drive/MyDrive/CS546/course-project/SemEval/data/data_en_subtask2/train-articles-subtask-2"
folder_dev = "/content/drive/MyDrive/CS546/course-project/SemEval/data/data_en_subtask2/dev-articles-subtask-2"
labels_train_fn = "/content/drive/MyDrive/CS546/course-project/SemEval/data/data_en_subtask2/train-labels-subtask-2.txt"

out_fn = "out.csv"

#Read Data
print('Loading training...')
train = make_dataframe(folder_train, labels_train_fn)
print('Loading dev...')
test = make_dataframe(folder_dev)

# the rest of the code is done with SVM baseline

# X_train = train['text'].values
# X_test = test['text'].values

# encoder = MultiLabelBinarizer() #use sklearn binarizer

# Y_train = train['frames'].str.split(',').values
# Y_train = encoder.fit_transform(Y_train)


# pipe = Pipeline([('vectorizer',CountVectorizer(ngram_range = (1, 2), 
#                                             analyzer='word')),
#             ('SVM_multiclass', MultiOutputClassifier(svm.SVC(class_weight= None, C=1, kernel='linear'),n_jobs=1))])

# pipe.fit(X_train,Y_train)

# print('In-sample Acc: \t\t', pipe.score(X_train,Y_train))

# Y_pred = pipe.predict(X_test)
# out = encoder.inverse_transform(Y_pred)
# out = list(map(lambda x: ','.join(x), out))
# out = pd.DataFrame(out, test.index)
# out.to_csv(out_fn, sep='\t', header=None)
# print('Results on: ', out_fn)

Loading training...


433it [00:03, 130.45it/s]


Loading dev...


83it [00:17,  4.82it/s]


In-sample Acc: 		 0.9953810623556582
Results on:  out.csv


In [8]:
train

,text,frames
id,,
833042063,Chelsea Handler Admits She’s ‘Very Sexually At...,"Political,Morality,Fairness_and_equality,Exter..."
832959523,How Theresa May Botched\n\nThose were the time...,"Morality,Security_and_defense,Policy_prescript..."
833039623,Robert Mueller III Rests His Case—Dems NEVER W...,"Political,Crime_and_punishment,External_regula..."
833032367,Robert Mueller Not Recommending Any More Indic...,"Political,Crime_and_punishment,Fairness_and_eq..."
814777937,The Far Right Is Trying to Co-opt the Yellow V...,"Political,Morality,Fairness_and_equality,Exter..."
...,...,...
832908978,Trump Kept Asking Friends If the Mueller Repor...,"Political,Crime_and_punishment,External_regula..."
832910505,"The Mueller Circus is Over, Long Live the Muel...","Political,Morality,Crime_and_punishment,Qualit..."
832917532,Robert Mueller Not Recommending Any More Indic...,"Crime_and_punishment,Political,Legality_Consti..."


In [9]:
test

,text
id,
820791520,George III Lost America.\n\nTheresa May Could ...
828866387,Brexit in 23 days: EU says still 'no solution'...
821040551,Queen Elizabeth Would Be Evacuated in Event of...
813552066,"You insult us, ambassador: Woody Johnson flagr..."
817176202,"The British People, as Well as the Politicians..."
...,...
829815104,Brussels Shows Its Fear\n\nHungarian Prime Min...
817147979,BREXIT OR BRINO: U.K.\n\nDeep State Strikes Ba...
813623212,'Wishful thinking': Tory MPs dismiss May's hop...


In [10]:
train.to_csv("/content/drive/MyDrive/CS546/course-project/SemEval/data/data_en_subtask2/train.csv")
test.to_csv("/content/drive/MyDrive/CS546/course-project/SemEval/data/data_en_subtask2/test.csv")

In [11]:
# sub task 3

def make_dataframe_st3(input_folder, labels_fn=None):
    #MAKE TXT DATAFRAME
    text = []
    for fil in tqdm(filter(lambda x: x.endswith('.txt'), os.listdir(input_folder))):
        iD = fil[7:].split('.')[0]
        lines = list(enumerate(open(input_folder+fil,'r',encoding='utf-8').read().splitlines(),1))
        text.extend([(iD,) + line for line in lines])

    df_text = pd.DataFrame(text, columns=['id','line','text'])
    df_text.id = df_text.id.apply(int)
    df_text.line = df_text.line.apply(int)
    df_text = df_text[df_text.text.str.strip().str.len() > 0].copy()
    df_text = df_text.set_index(['id','line'])
    
    df = df_text

    if labels_fn:
        #MAKE LABEL DATAFRAME
        labels = pd.read_csv(labels_fn,sep='\t',encoding='utf-8',header=None)
        labels = labels.rename(columns={0:'id',1:'line',2:'labels'})
        labels = labels.set_index(['id','line'])
        labels = labels[labels.labels.notna()].copy()

        #JOIN
        df = labels.join(df_text)[['text','labels']]

    return df

In [ ]:
folder_train = "/content/drive/MyDrive/CS546/course-project/SemEval/data/data_en_subtask2/train-articles-subtask-2"
folder_dev = "/content/drive/MyDrive/CS546/course-project/SemEval/data/data_en_subtask2/dev-articles-subtask-2"
labels_train_fn = "/content/drive/MyDrive/CS546/course-project/SemEval/data/data_en_subtask2/train-labels-subtask-2.txt"

out_fn = "out.csv"

#Read Data
print('Loading training...')
train = make_dataframe(folder_train, labels_train_fn)
print('Loading dev...')
test = make_dataframe(folder_dev)